In [1]:
#baseDIR = '/home/pataki/synapse/gitParkinson/' # base directory of the github repo
#uncomment the line above (+ update) if you are running this notebook in an empty namespace
try: baseDIR
except NameError:
    print('Error: baseDIR not found!')

In [2]:
cd $baseDIR/sub2.2_dysk/

/home/pataki/synapse/gitParkinson/sub2.2_dysk


In [3]:
%run -i ../src/helperFuncs.py

In [4]:
%run -i ../src/sc2FitModels.py

Welcome, Balint Armin Pataki!



In [5]:
%matplotlib inline

In [6]:
mainDF = mainDFCreator('dyskinesiaScore')
mainDF = mainDFtrimmer(fileMinLen=100, inputDF=mainDF, plot=False)

Train shape: (1556, 12)
Test shape:  (660, 12)
Merged:      (2216, 12)
Remained shape: (2142, 14)


In [7]:
xRange = [] # difference between the 80% and 20% percentile
yRange = []
zRange = []
for i in pb(range(len(mainDF))):
    tmp = dataByID(mainDF.dataFileHandleId.tolist()[i], mainDF).describe(percentiles=[0.2, 0.8]).T
    xRange.append(np.abs(tmp['80%'] - tmp['20%']).X)
    yRange.append(np.abs(tmp['80%'] - tmp['20%']).Y)
    zRange.append(np.abs(tmp['80%'] - tmp['20%']).Z)
    
mainDF['xR'] = xRange
mainDF['yR'] = yRange
mainDF['zR'] = zRange

100%|██████████| 2142/2142 [00:34<00:00, 32.17it/s]


In [8]:
xStd = [] # standard deviation for the values between the 90% and 10% percentile
yStd = []
zStd = []
for i in pb(range(len(mainDF))):
    rawData = dataByID(mainDF.dataFileHandleId.tolist()[i], mainDF)
    tmp     = rawData.describe(percentiles=[0.2, 0.8]).T
    xStd.append(rawData[(rawData.X < tmp['80%'].X) & (rawData.X > tmp['20%'].X)].X.std())
    yStd.append(rawData[(rawData.Y < tmp['80%'].Y) & (rawData.Y > tmp['20%'].Y)].Y.std())
    zStd.append(rawData[(rawData.Z < tmp['80%'].Z) & (rawData.Z > tmp['20%'].Z)].Z.std())
    
mainDF['xStd'] = xStd
mainDF['yStd'] = yStd
mainDF['zStd'] = zStd

100%|██████████| 2142/2142 [00:40<00:00, 25.41it/s]


In [9]:
mainDF[['xR', 'yR', 'zR', 'xStd', 'yStd', 'zStd', 
        'dataFileHandleId']].to_csv('featureDB/rangeStdFeatures.tsv', sep='\t', index=False)

# Take autocorrelation function as a function of lag. Then take the power spectra as feature.

In [10]:
parityInv = {0:1, 1:0} # sampling for Fourier odd/even
autoFourierDF = pd.DataFrame()

for ID in pb(mainDF.dataFileHandleId.tolist()):
    XautoCorrVals = [] # autocorrelation function
    YautoCorrVals = []
    ZautoCorrVals = []

    rawData = dataByID(ID, mainDF)
    for i in range(len(rawData) - 40):
        XautoCorrVals.append(rawData.X.autocorr(20 + i))
        YautoCorrVals.append(rawData.Y.autocorr(20 + i))
        ZautoCorrVals.append(rawData.Z.autocorr(20 + i))
    fileLen = len(rawData) - 40
    timeScale = np.fft.rfftfreq(d=0.001, n = fileLen)
    xF = np.abs(np.fft.rfft(XautoCorrVals))**2 # Fourier transform of the autocorrelation function
    yF = np.abs(np.fft.rfft(YautoCorrVals))**2
    zF = np.abs(np.fft.rfft(ZautoCorrVals))**2

    tmpDF = pd.DataFrame({'timeScale':timeScale, 'xAutoFourier': xF, 'yAutoFourier': yF, 'zAutoFourier': zF})
    tmpDF = tmpDF[tmpDF.timeScale <= 10.0]
    tmpDF['timeScale'] = [int(i) for i in tmpDF.timeScale] # groupby freq (1 hz bins)
    tmpDF = tmpDF.groupby('timeScale').mean()

    allPowDF = pd.DataFrame(tmpDF.sum(0)).rename(columns = {0:'allPower'}).T # sum of the power spectra
    tmpDF = tmpDF.div(tmpDF.sum(0)).append(allPowDF)

    xDF = tmpDF[['xAutoFourier']].T
    xDF.columns = ['xAutoFourier_' + str(i) for i in xDF.columns.tolist()]

    yDF = tmpDF[['yAutoFourier']].T
    yDF.columns = ['yAutoFourier_' + str(i) for i in yDF.columns.tolist()]

    zDF = tmpDF[['zAutoFourier']].T
    zDF.columns = ['zAutoFourier_' + str(i) for i in zDF.columns.tolist()]

    allDF = pd.concat([xDF.reset_index(drop=True), yDF.reset_index(drop=True)], axis=1)
    allDF = pd.concat([allDF, zDF.reset_index(drop=True)], axis = 1)

    allDF['dataFileHandleId'] = ID
    
    autoFourierDF = autoFourierDF.append(allDF)

100%|██████████| 2142/2142 [39:44<00:00,  1.24s/it]


## There are some too short files. For them I have NaN-s. Fill them up!

In [11]:
mainDF = mainDFCreator('dyskinesiaScore')
mainDF = mainDFtrimmer(fileMinLen=1, inputDF=mainDF, plot=False)

Train shape: (1556, 12)
Test shape:  (660, 12)
Merged:      (2216, 12)
Remained shape: (2143, 14)


In [12]:
fourierDF = pd.merge(autoFourierDF.fillna(autoFourierDF.mean()), mainDF[['dataFileHandleId']], how = 'outer')

In [13]:
fourierDF = fourierDF.fillna(fourierDF.mean())

In [14]:
fourierDF['xAutoFourier_allPower'] = np.log(fourierDF.xAutoFourier_allPower.tolist())
fourierDF['yAutoFourier_allPower'] = np.log(fourierDF.yAutoFourier_allPower.tolist())
fourierDF['zAutoFourier_allPower'] = np.log(fourierDF.zAutoFourier_allPower.tolist())

In [15]:
fourierDF.fillna(fourierDF.mean()).to_csv('featureDB/autoCorrFeatures.tsv', sep='\t', index=False)